In [1]:
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
import pandas as pd
import numpy as np
breast_cancer_df = fetch_ucirepo(id=14)  
X = pd.DataFrame(breast_cancer_df.data.features, columns=breast_cancer_df.data.feature_names)
y = pd.Series(breast_cancer_df.data.targets.squeeze(), name='Outcome')
columns_to_one_hot = [col for col in X.columns if X[col].dtype == 'object']
X = pd.get_dummies(X, columns=columns_to_one_hot)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_encoded = pd.get_dummies(X)
#change from bool to 1 and 2 without independently mapping everything.
breast_cancer_df = X_encoded.select_dtypes(include=['bool']).columns
for col in breast_cancer_df:
    X_encoded[col] = X_encoded[col].map({False: 1, True: 2})

# SVM

classifer = make_pipeline(StandardScaler(), OneClassSVM(gamma='auto'))
classifer.fit(X_train)
y_pred_svm = classifer.predict(X_test)
pred = pd.DataFrame({'svm': y_pred_svm})
pred['svm_pred'] = np.where(pred['svm'] == -1, 1, 0)
y_pred_svm = pred['svm_pred']
precision_macro = precision_score(y_test, y_pred_svm, average='macro')
print("OneClassSVM Macro Precision:", precision_macro)